# Mecanismos de atención

En este notebook, exploraremos cómo resolver un problema de análisis de sentimientos utilizando mecanismos de **self-attention**.

El mecanismo de self-attention permite que cada elemento de una secuencia preste atención a otros elementos de la misma secuencia. Esto es útil para capturar dependencias a **largo plazo** a diferencia de las redes recurrentes tradicionales.

Además, a diferencia de las redes LSTM o GRU, estas son **altamente paralelizables** dado que permiten tratar cada elemento de la secuencia de forma individual.


## Conjunto de datos

Utilizaremos un conjunto de datos pensado para la tarea de análisis de sentimientos. En este caso hemos optado por un conjunto de twits escritos en inglés, los cuales ya vienen etiquetados en tres clases:
* Positive
* Neutral
* Negative

In [ ]:
import pandas as pd
data_url = "https://raw.githubusercontent.com/pablo-pnunez/datasets/master/texto/sentiment_analysis_twitter.csv"
twitter_data = pd.read_csv(data_url)
tweets, tweets_sentiment = twitter_data["text"].str.strip().values, twitter_data["sentiment"].values

### Preprocesar texto

Normalmente los conjuntos de datos contienen errores o elementos no deseados, en esta parte los eliminaremos. 

Algunos ejemplos típicos son los números, los saltos de línea `\n` o los tabuladores `\t`.

En esta parte separaremos además las entradas y salidas esperadas del modelo.

In [ ]:
! pip install unidecode

In [ ]:
import re
import time
import nltk
from unidecode import unidecode

x_data = []
y_data = []

for idx, tweet in enumerate(tweets):
    try:
        # Eliminar saltos de línea y retornos de carro
        tweet = re.sub(r'[\n\r]+', ' ', tweet)
        # Eliminar acentos utilizando la librería unidecode
        tweet = unidecode(tweet)
        # Pasar a minúsculas y eliminar resto de caracteres extraños (dos puntos, punto y coma, números...)
        tweet = re.sub(r'[^a-z\s]', '', tweet.lower())
        # Eliminamos más de un espacio
        tweet = re.sub(r'(\s)+', ' ', tweet)
        # Filtrar secuencias de 1 palabra o menos
        if len(tweet.split()) > 1:
            x_data.append(tweet)
            y_data.append(tweets_sentiment[idx])
    except:
        print(f"Error con el ejemplo {idx}, no se añade.")

### Obtener vocabulario

Una vez tenemos todas las frases, para codificar cada una de las palabras del texto es necesario obtener el llamado `corpus` o `vocabulario`. Este nos sirve para saber que palabras únicas (sin repetición) aparecen en nuestro texto, lo cual es útil para asignar un índice a cada una. Como verás, en este caso asignamos el índice en función de la frecuencia de aparición, por tanto, la palabra más frecuente tendrá el índice más bajo.

Como recordarás, nuestro objetivo es que, dado un texto de máximo 5 palabras, el modelo entrenado sea capaz de continuar la frase hasta que decida finalizarla. Para lograrlo tenemos que tener en cuenta varios escenarios:
* ¿Y si nos dan menos de 5 palabras?
* ¿Como hacemos para detectar que la palabra generada por el modelo es la última de la frase? ¿Y la primera?
* ¿Y si nos dan palabras fuera del vocabulario?

Para solventar estos problemas, vamos a añadir dos palabras nuevas o `tokens` nuevos a nuestro vocabulario:
* **\<pad\>**: Token que representa *padding*, es decir, si nos dan menos de 5 palabras, rellenaremos las que falten con este padding.
* **\<unk\>**: Token que representa *unknown*, es decir, palabra desconocida (fuera del vocabulario).
* **\<eos\>**: Token que representa *end-of-sequence*, es decir, fin de secuencia. Si el modelo predice este token, sabremos que ya se acabó la frase y podemos detener la generación.
* **\<sos\>**: Token que representa *start-of-sequence*, es decir, comienzo de secuencia.


In [ ]:
from collections import Counter

# Crear una lista con todas las palabras del texto
words = " ".join(x_data).split()
# Obtener el corpus del texto (todas las palabras que aparecen y su frecuencia)
word_counts = Counter(words)
# Filtrar palabras que aparecen menos de 5 veces, dado que no aportan mucho
word_counts = {word: count for word, count in word_counts.items() if count >=5}
# Ordenamos el corpus por frecuencia
word_counts = dict(sorted(word_counts.items(), key=lambda x: x[1], reverse=True))
# Extraemos solo las claves del diccionario anterior para crear el corpus
vocab = list(word_counts.keys())
# Añadimos tokens especiales <pad> (padding) y <unk> (unknown)
vocab = ["<pad>", "<unk>", "<sos>", "<eos>"] + vocab
# Generamos un diccionario (y su inverso) donde asignamos un id a cada palabra según su frecuencia.
# La palabra más frecuente será la 1. El 0 lo dejamos para el padding
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)

print(f'Tamaño del vocabulario: {vocab_size}')
print(f'Word to Index mapping: {word_to_ix}')

### Crear dataset
Una vez tenemos nuestro conjunto limpio y analizado, podemos crear los ejemplos con los que alimentaremos nuestro modelo. 

En este caso simplemente tendremos que buscar la frase de mayor longitud y crear todos los vectores de ese tamaño rellenando con padding.

Tendremos que crear ademas las máscaras y los vectores de salida.

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
# Generar dataset completo
def generate_dataset(sentences, labels, word_to_ix, seq_length, y_map):
    # Para cada frase, generar los ejemplos
    x_data = []
    mask_data = []
    y_data = []

    for sentence, label in zip(sentences, labels):
        unk_index = word_to_ix["<unk>"]
        words = sentence.split()

        # Crear la secuencia de entrada
        seq_in = ['<pad>'] * (seq_length - len(words)) + words
        seq_in_indices = [word_to_ix.get(word, unk_index) for word in seq_in]
        x_data.append(seq_in_indices)

        # Creamos la máscara de padding
        mask = [1 if word != '<pad>' else 0 for word in seq_in]
        mask_data.append(mask)
        y_data.append(y_map[label])

    return tf.data.Dataset.from_tensor_slices(((x_data, mask_data), y_data))

# Dividir el dataset en entrenamiento, validación y prueba (80%,10%,10%)
sentences_train, sentences_test, labels_train, labels_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
sentences_val, sentences_test, labels_val, labels_test = train_test_split(sentences_test, labels_test, test_size=0.5, random_state=42)

# Definir parámetros y datos
max_seq_length = max([len(s.split(" ")) for s in x_data])
y_map = {"positive":0, "neutral":1, "negative":2}
batch_size = 1024

# Generar dataset
train_dataset = generate_dataset(sentences_train, labels_train, word_to_ix, max_seq_length, y_map).batch(batch_size)
val_dataset = generate_dataset(sentences_val, labels_val, word_to_ix, max_seq_length, y_map).batch(batch_size)
test_dataset = generate_dataset(sentences_test, labels_test, word_to_ix, max_seq_length, y_map).batch(batch_size)

## 4.2.3. - Modelo
Para este problema crearemos el modelo más sencillo posible:
* **Capa Embedding**: Para cada elemento de la entrada, aprenderá un embedding que lo representará en un espacio de dimensión `embed_size`.
* **Capa MultiheadAttention**: Esta capa permite al modelo enfocarse en diferentes partes de la secuencia de entrada simultáneamente. Cada "cabeza" en la atención múltiple (multi-head) puede capturar distintos patrones de relación entre los elementos de la secuencia.
    * En la capa LSTM de la práctica anterior, el último elemento de la secuencia de salida contiene información de toda la secuencia, **en este caso no**.
    * El número de heads se configura mediante `num_heads`.
* **Capa Linear**: Proyecta el vector del último elemento de la salida de la LSTM en un espacio de tamaño `output_size`. En este caso `output_size` es igual al tamaño del vocabulario dado que estamos intentando resolver un problema de multi-clasificación.

Nótese que no aplicamos la función `softmax` dado que esta se aplica internamente en la `CrossEntropyLoss`.

In [ ]:
from tensorflow.keras import layers, Model, Input

# Hiperparámetros del modelo
embed_size = 32
num_heads = 1  # Número de cabezas de atención
hidden_size = 16
output_size = 3

num_epochs = 100
learning_rate = 0.0005

# Definir el modelo usando la API funcional
input_seq = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_seq')
input_mask = Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')

# Esta parte adapta la máscara a lo requerido por la capa attention
expanded_mask = layers.Lambda(lambda x: tf.repeat(tf.expand_dims(x, axis=1), max_seq_length, axis=1))(input_mask)

embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=max_seq_length)(input_seq)
dropout_1 = layers.Dropout(.5)(embedding_layer)
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=hidden_size)(dropout_1, dropout_1, attention_mask=expanded_mask)
dropout_2 = layers.Dropout(.5)(attention_output)
flatten_output = layers.GlobalAveragePooling1D()(dropout_2)
output_layer = layers.Dense(output_size, activation="softmax")(flatten_output)
model = Model(inputs=[input_seq, input_mask], outputs=output_layer)

# Compilar el modelo (necesario en TensorFlow/Keras)
model.compile(optimizer=tf.optimizers.Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()

## Entrenamiento

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs)

## Evaluación

In [ ]:
# Definir la función de predicción
def predict(model, text, word_to_ix, ix_to_word, seq_length, y_map):
    # Preprocesar el texto de entrada
    text = unidecode(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()

    # Convertir palabras a índices
    input_sequence = [word_to_ix.get(word, word_to_ix['<pad>']) for word in words]
    input_sequence = input_sequence[-seq_length:]  # Tomar solo las últimas seq_length palabras
    input_sequence = [word_to_ix['<pad>']] * (seq_length - len(input_sequence)) + input_sequence


    # Convertir la secuencia y la máscara a tensores
    input_tensor = tf.constant([input_sequence], dtype=tf.int64)
    mask_tensor = tf.cast(input_tensor!=0, tf.int32)


    output = model([input_tensor, mask_tensor], training=False)
    
    max_output = output.numpy().argmax()
    max_pred = output.numpy().max()

    print(f"The text '{text}' is {list(y_map.keys())[max_output]} [{max_pred*100:0.0f}%]")

# Ejemplo de texto de entrada
input_text = "This product is lovely"
predicted_text = predict(model, input_text, word_to_ix, ix_to_word, max_seq_length, y_map)